In [ ]:
import torch
from transformers import TFAutoModelForSequenceClassification
from transformers import BertTokenizerFast
#import datasets

In [ ]:
import numpy as np
import pandas as pd
#import evaluate

In [ ]:
import tensorflow as tf

Загружаем токенайзер и объект модели предобученной модели rubert

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = TFAutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTo

Карта преобразования меток: метки в модели отличаются от наших данных

In [ ]:
LABELS = {
    0: 'NEUTRAL',
    1: 'POSITIVE',
    2: 'NEGATIVE',
}

TABIT_LABELS = {
    0: 'BAD',
    1: 'NEUTRAL',
    2: 'GOOD'
}

ADJUST_MAP = {
    0: 2,
    1: 0,
    2: 1,
}

def adjust(label):
  return ADJUST_MAP[label]

Загружаем данные, собранные на tabiturient.ru

In [ ]:
reviews = pd.read_json('/content/drive/MyDrive/Data/tabiturient/unique-reviews.jsonl', lines=True)

In [ ]:
review_texts=reviews['content'].to_list()

In [ ]:
review_labels=reviews['label'].map(adjust).to_list()

In [ ]:
train_texts=review_texts[0:3235]
valid_texts=review_texts[3235:]

In [ ]:
train_labels=review_labels[0:3235]
valid_labels=review_labels[3235:]

Векторизуем текст

In [ ]:
train_inputs = tokenizer(train_texts, max_length=512, padding=True, truncation=True, return_tensors='tf')
valid_inputs = tokenizer(valid_texts, max_length=512, padding=True, truncation=True, return_tensors='tf')


Вычисляем предсказания модели

In [ ]:
outputs=model.predict(train_inputs)

102/102 [==============================] - 158s 1s/step


Применяем активацию softmax

In [ ]:
layer=tf.keras.layers.Activation('softmax')

In [ ]:
 predicted_lables = layer(outputs.logits)

In [ ]:
predicted_lables

<tf.Tensor: shape=(3235, 3), dtype=float32, numpy=
array([[0.18069902, 0.06785828, 0.7514427 ],
       [0.1806979 , 0.06785849, 0.7514436 ],
       [0.18069918, 0.06785779, 0.751443  ],
       ...,
       [0.01567174, 0.98058045, 0.0037478 ],
       [0.03443173, 0.9563452 , 0.00922298],
       [0.6298857 , 0.3091212 , 0.06099319]], dtype=float32)>

Сравниваем предсказанные и эталонные оценки метрикой CategoricalAccuracy

In [ ]:
metric = tf.keras.metrics.CategoricalAccuracy()

In [ ]:
train_labels_list_cat = tf.keras.utils.to_categorical(train_labels)
valid_labels_list_cat = tf.keras.utils.to_categorical(valid_labels)


In [ ]:
metric.update_state(predicted_lables, review_labels_list_cat)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=3235.0>

In [ ]:
print(f"Accuracy={metric.result().numpy()}" )

Accuracy=0.656877875328064


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(3e-5))

In [ ]:
model.fit(valid_inputs, valid_labels_list_cat, batch_size=16)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ResourceExhaustedError: Graph execution error:

Detected at node tf_bert_for_sequence_classification/bert/encoder/layer_._10/output/dropout_33/dropout/random_uniform/RandomUniform defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/pyt